In [1]:
# Importing the endpoints from the NBA API  (from https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints_data_structure.md)

from nba_api.stats.endpoints import leaguedashplayerstats, playerestimatedmetrics
import pandas as pd

# Using the parameter to set the needed season
season = '2023-24'

# Get player stats for the specified season from the leaguedashplayerstats endpoint
player_stats_dash = leaguedashplayerstats.LeagueDashPlayerStats(season=season)
player_stats_dash_df = player_stats_dash.get_data_frames()[0]

# Get player estimated metrics for the specified season from the playerestimatedmetrics endpoint
player_stats_estimated_metrics = playerestimatedmetrics.PlayerEstimatedMetrics(season=season)
player_stats_estimated_metrics_df = player_stats_estimated_metrics.get_data_frames()[0]

In [2]:
# Merge the two tables based on the common identifier
merged_df = pd.merge(player_stats_dash_df, player_stats_estimated_metrics_df, on='PLAYER_ID', how='inner')

# Removing the duplicate columns we dont't need
original_columns_to_drop = ['PLAYER_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK']
columns_to_drop = [col + '_y' for col in original_columns_to_drop]
merged_df = merged_df.drop(columns_to_drop, axis=1)


df = merged_df

In [3]:
df.columns

Index(['PLAYER_ID', 'PLAYER_NAME_x', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'AGE', 'GP_x', 'W_x', 'L_x', 'W_PCT_x', 'MIN_x',
       'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA',
       'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3',
       'WNBA_FANTASY_PTS', 'GP_RANK_x', 'W_RANK_x', 'L_RANK_x', 'W_PCT_RANK_x',
       'MIN_RANK_x', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK', 'E_OFF_RATING', 'E_DEF_RATING', 'E_NET_RATING',
       'E_AST_RATIO', 'E_OREB_PCT', 'E_DREB_PCT', 'E_REB_PCT', 'E_TOV_PCT',
       'E_USG_PCT', 'E_PACE', 'E_OF

In [4]:
# Dropping a list of columns not needed for this analysis

drop_list = [
    'E_OFF_RATING_RANK', 'E_DEF_RATING_RANK', 'E_NET_RATING_RANK',
    'E_AST_RATIO_RANK', 'E_OREB_PCT_RANK', 'E_DREB_PCT_RANK',
    'E_REB_PCT_RANK', 'E_TOV_PCT_RANK', 'E_USG_PCT_RANK', 'E_PACE_RANK',
    'GP_RANK_x', 'W_RANK_x', 'L_RANK_x', 'W_PCT_RANK_x', 'MIN_RANK_x',
    'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
    'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
    'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
    'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK',
    'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK', 'NICKNAME', 'PLAYER_ID', 'TEAM_ID',
    'WNBA_FANTASY_PTS_RANK', 'E_PACE'
]


df = df.drop(drop_list, axis=1)


df.columns


Index(['PLAYER_NAME_x', 'TEAM_ABBREVIATION', 'AGE', 'GP_x', 'W_x', 'L_x',
       'W_PCT_x', 'MIN_x', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
       'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL',
       'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS',
       'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'E_OFF_RATING', 'E_DEF_RATING',
       'E_NET_RATING', 'E_AST_RATIO', 'E_OREB_PCT', 'E_DREB_PCT', 'E_REB_PCT',
       'E_TOV_PCT', 'E_USG_PCT'],
      dtype='object')

In [5]:
# Calculate per game stats
per_game_stats_columns = [
    'FGM', 'FGA', 'FG3M', 'FG3A',
    'FTM', 'FTA', 'OREB', 'DREB', 'REB',
    'AST', 'TOV', 'STL', 'BLK', 'PF', 'PFD', 'PTS',
    'MIN_x', 'BLKA'
]

for column in per_game_stats_columns:
    df[column] = df[column] / df['GP_x']

# Display the per game stats
df.sort_values(by="PTS",ascending=False).head()


,PLAYER_NAME_x,TEAM_ABBREVIATION,AGE,GP_x,W_x,L_x,W_PCT_x,MIN_x,FGM,FGA,...,WNBA_FANTASY_PTS,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,E_AST_RATIO,E_OREB_PCT,E_DREB_PCT,E_REB_PCT,E_TOV_PCT,E_USG_PCT
262,Joel Embiid,PHI,29.0,34,26,8,0.765,34.025441,11.823529,22.176471,...,2017.0,118.8,109.2,9.7,15.4,0.088,0.270,0.182,10.077,0.392
348,Luka Doncic,DAL,24.0,43,25,18,0.581,37.684147,11.651163,23.906977,...,2605.0,117.5,116.2,1.4,22.7,0.023,0.238,0.130,9.546,0.364
173,Giannis Antetokounmpo,MIL,29.0,49,32,17,0.653,35.437551,11.714286,19.163265,...,2668.0,120.2,112.9,7.3,18.6,0.084,0.264,0.177,10.594,0.329
470,Shai Gilgeous-Alexander,OKC,25.0,50,35,15,0.700,34.579233,10.900000,19.940000,...,2535.0,120.4,109.2,11.3,20.1,0.031,0.145,0.090,6.669,0.323
141,Donovan Mitchell,CLE,27.0,40,27,13,0.675,35.549625,10.050000,21.200000,...,1931.0,116.0,104.7,11.3,19.2,0.031,0.133,0.083,8.500,0.320


In [6]:
# Renaming the columns to add the per game stats

column_rename = {
    'PLAYER_NAME_x': 'PlayerName',
    'TEAM_ABBREVIATION': 'Team',
    'AGE': 'Age',
    'GP_x': 'GP',
    'W_x': 'W',
    'L_x': 'L',
    'W_PCT_x': 'W%',
    'MIN_x': 'MIN_PG',
    'FGM': 'FGM_PG',
    'FGA': 'FGA_PG',
    'FG_PCT': 'FG%',
    'FG3M': '3PM_PG',
    'FG3A': '3PA_PG',
    'FG3_PCT': '3P%',
    'FTM': 'FTM_PG',
    'FTA': 'FTA_PG',
    'FT_PCT': 'FT%',
    'OREB': 'OREB_PG',
    'DREB': 'DREB_PG',
    'REB': 'TREB_PG',
    'AST': 'AST_PG',
    'TOV': 'TO_PG',
    'STL': 'STL_PG',
    'BLK': 'BLK_PG',
    'BLKA': 'BLKA_PG',
    'PF': 'PF_PG',
    'PFD': 'PFD_PG',
    'PTS': 'PPG',
    'PLUS_MINUS': 'PlusMinus',
    'NBA_FANTASY_PTS': 'NBAFantasyPoints',
    'DD2': 'DoubleDoubles',
    'TD3': 'TripleDoubles',
    'WNBA_FANTASY_PTS': 'WNBAFantasyPoints',
    'E_OFF_RATING': 'OFF_Rating',
    'E_DEF_RATING': 'DEF_Rating',
    'E_NET_RATING': 'Net_Rating',
    'E_AST_RATIO': 'AST_Ratio',
    'E_OREB_PCT': 'OREB%',
    'E_DREB_PCT': 'DREB%',
    'E_REB_PCT': 'TREB%',
    'E_TOV_PCT': 'TOV%',
    'E_USG_PCT': 'USG%'
}


df = df.rename(columns=column_rename)
pd.set_option('display.max_columns', None)
df.head(10)

,PlayerName,Team,Age,GP,W,L,W%,MIN_PG,FGM_PG,FGA_PG,FG%,3PM_PG,3PA_PG,3P%,FTM_PG,FTA_PG,FT%,OREB_PG,DREB_PG,TREB_PG,AST_PG,TO_PG,STL_PG,BLK_PG,BLKA_PG,PF_PG,PFD_PG,PPG,PlusMinus,NBAFantasyPoints,DoubleDoubles,TripleDoubles,WNBAFantasyPoints,OFF_Rating,DEF_Rating,Net_Rating,AST_Ratio,OREB%,DREB%,TREB%,TOV%,USG%
0,A.J. Lawson,DAL,23.0,23,11,12,0.478,9.474420,1.608696,3.434783,0.468,0.521739,1.652174,0.316,0.521739,0.826087,0.632,0.434783,0.782609,1.217391,0.565217,0.434783,0.391304,0.130435,0.260870,0.739130,0.478261,4.260870,33,177.1,0,0,175.0,115.3,108.4,7.0,11.8,0.043,0.090,0.065,9.061,0.178
1,AJ Green,MIL,24.0,31,22,9,0.710,8.093280,1.161290,2.741935,0.424,0.967742,2.387097,0.405,0.161290,0.161290,1.000,0.161290,0.741935,0.903226,0.516129,0.032258,0.000000,0.032258,0.032258,0.870968,0.225806,3.451613,33,166.6,0,0,183.0,123.0,115.2,7.7,15.4,0.025,0.095,0.063,0.960,0.154
2,AJ Griffin,ATL,20.0,18,8,10,0.444,7.360000,0.722222,2.500000,0.289,0.500000,1.833333,0.273,0.111111,0.111111,1.000,0.111111,0.666667,0.777778,0.222222,0.333333,0.055556,0.055556,0.111111,0.333333,0.055556,2.055556,-28,59.8,0,0,68.0,111.5,120.0,-8.6,7.2,0.017,0.112,0.062,10.737,0.165
3,Aaron Gordon,DEN,28.0,45,30,15,0.667,31.883037,5.377778,9.911111,0.543,0.555556,1.822222,0.305,2.311111,3.666667,0.630,2.600000,4.022222,6.622222,3.133333,1.333333,0.755556,0.666667,0.733333,1.933333,3.111111,13.622222,229,1314.1,9,0,1205.0,118.5,111.6,6.9,19.6,0.092,0.139,0.116,8.338,0.175
4,Aaron Holiday,HOU,27.0,47,22,25,0.468,17.085887,2.297872,5.106383,0.450,1.127660,2.808511,0.402,0.723404,0.787234,0.919,0.255319,1.574468,1.829787,1.787234,0.787234,0.553191,0.063830,0.212766,1.510638,0.808511,6.446809,72,582.2,0,0,584.0,110.6,106.9,3.7,22.3,0.016,0.099,0.058,9.807,0.154
5,Aaron Nesmith,IND,24.0,47,27,20,0.574,27.099043,4.574468,8.744681,0.523,2.148936,4.638298,0.463,1.234043,1.723404,0.716,0.978723,2.978723,3.957447,1.468085,0.957447,1.042553,0.659574,0.702128,3.382979,1.914894,12.531915,60,1110.7,1,0,1105.0,119.2,116.8,2.5,12.3,0.043,0.125,0.085,8.027,0.163
6,Aaron Wiggins,OKC,25.0,48,34,14,0.708,14.127986,2.437500,4.145833,0.588,0.791667,1.520833,0.521,0.520833,0.708333,0.735,0.708333,1.666667,2.375000,0.958333,0.541667,0.458333,0.104167,0.354167,1.187500,0.583333,6.187500,32,557.8,0,0,549.0,116.5,109.9,6.6,16.1,0.060,0.128,0.096,9.092,0.159
7,Adama Sanogo,CHI,21.0,3,0,3,0.000,3.537222,1.000000,1.000000,1.000,0.000000,0.000000,0.000,0.666667,1.000000,0.667,0.666667,1.333333,2.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.666667,2.666667,17,14.2,0,0,14.0,85.6,8.3,77.3,0.0,0.167,0.500,0.300,18.797,0.203
8,Admiral Schofield,ORL,26.0,12,6,6,0.500,3.846944,0.416667,1.250000,0.333,0.166667,0.750000,0.222,0.000000,0.166667,0.000,0.083333,0.500000,0.583333,0.333333,0.333333,0.000000,0.000000,0.166667,0.583333,0.250000,1.000000,9,22.4,0,0,25.0,105.0,90.5,14.5,16.8,0.022,0.140,0.080,16.750,0.194
9,Al Horford,BOS,37.0,42,32,10,0.762,27.130476,2.976190,6.214286,0.479,1.690476,4.166667,0.406,0.285714,0.333333,0.857,1.261905,5.547619,6.809524,2.952381,0.785714,0.523810,1.023810,0.119048,1.547619,0.380952,7.928571,258,1024.2,3,0,944.0,118.0,107.5,10.6,29.2,0.053,0.209,0.135,7.780,0.115


In [7]:
# Let's round up the stats to 1 decimal place

df['MIN_PG'] = df['MIN_PG'].round(1)
df['FGM_PG'] = df['FGM_PG'].round(1)
df['FGA_PG'] = df['FGA_PG'].round(1)
df['3PM_PG'] = df['3PM_PG'].round(1)
df['3PA_PG'] = df['3PA_PG'].round(1)
df['FTM_PG'] = df['FTM_PG'].round(1)
df['FTA_PG'] = df['FTA_PG'].round(1)
df['OREB_PG'] = df['OREB_PG'].round(1)
df['DREB_PG'] = df['DREB_PG'].round(1)
df['TREB_PG'] = df['TREB_PG'].round(1)
df['AST_PG'] = df['AST_PG'].round(1)
df['TO_PG'] = df['TO_PG'].round(1)
df['STL_PG'] = df['STL_PG'].round(1)
df['BLK_PG'] = df['BLK_PG'].round(1)
df['BLKA_PG'] = df['BLKA_PG'].round(1)
df['PF_PG'] = df['PF_PG'].round(1)
df['PFD_PG'] = df['PFD_PG'].round(1)
df['PPG'] = df['PPG'].round(1)

df[["PlayerName","PPG"]].sort_values(by="PPG", ascending=False).head(10)


,PlayerName,PPG
262,Joel Embiid,35.3
348,Luka Doncic,34.5
173,Giannis Antetokounmpo,31.3
470,Shai Gilgeous-Alexander,31.1
141,Donovan Mitchell,28.5
314,Kevin Durant,28.3
133,Devin Booker,27.9
478,Stephen Curry,27.7
216,Jalen Brunson,27.2
500,Trae Young,27.1


In [8]:
df.shape

(535, 42)

In [9]:
df[["PlayerName","PPG"]].sort_values(by="PPG",ascending=False).head()

,PlayerName,PPG
262,Joel Embiid,35.3
348,Luka Doncic,34.5
173,Giannis Antetokounmpo,31.3
470,Shai Gilgeous-Alexander,31.1
141,Donovan Mitchell,28.5


In [10]:
# Parameter 1/5 : WINS - we are going to use a weighted avg between Wins and Wins%
# Win% matters the most as some players might miss days and rest for games against very weak teams

weightedW = 0.40
weightedW_p = 0.60

# We are focusing on players who played at least 20 games, shot at least 5 times per game and played at least 30 mins per game
df2 =  df[(df.FGA_PG >= 5) & (df.GP >= 20) & (df.MIN_PG >= 30)].copy()

df2["Win_Par"] = round((df["W"] * weightedW) * (df["W%"] * weightedW_p), 2)

df2[["PlayerName", "Team", "Win_Par"]].sort_values(by="Win_Par", ascending=False).head(10)

,PlayerName,Team,Win_Par
131,Derrick White,BOS,6.99
248,Jayson Tatum,BOS,6.85
288,Jrue Holiday,BOS,6.39
245,Jaylen Brown,BOS,6.26
470,Shai Gilgeous-Alexander,OKC,5.88
452,Rudy Gobert,MIN,5.88
72,Chet Holmgren,OKC,5.76
379,Michael Porter Jr.,DEN,5.76
406,Nikola Jokic,DEN,5.66
310,Kentavious Caldwell-Pope,DEN,5.66


In [11]:
# Parameter 2/5: OFFENSIVE - we are going to add some offensive stats together

# First we get an scoring efficiency stat bu multiplying PPG by FG%

df2['P/M'] = df['PPG'] * df["FG%"]

# We are going to add this to Offensive Rebounds per game and Assists per game, taking away Turnovers

df2['Off_Par'] = df2['P/M'] + df2['OREB_PG'] + df2['AST_PG'] - df2['TO_PG']

# Finally, we are going to find an average using the Offensive Rating, which is expressed in points scored per 100 possessions
# To make this stat usable, we are going to subtract 100 from the total
# This will produce a stat that takes in consideration offensive efficiency 

df2['OffRat_Par'] = ((df2['Off_Par'] + (df2['OFF_Rating'] - 100))/2)

df2[["PlayerName", "OffRat_Par", ]].sort_values(by="OffRat_Par", ascending=False).head(10).round(1)

,PlayerName,OffRat_Par
406,Nikola Jokic,22.7
173,Giannis Antetokounmpo,22.3
515,Tyrese Haliburton,21.6
470,Shai Gilgeous-Alexander,21.4
262,Joel Embiid,21.1
348,Luka Doncic,20.3
133,Devin Booker,19.6
216,Jalen Brunson,18.0
314,Kevin Durant,17.9
301,Kawhi Leonard,17.7


In [12]:
# Parameter 3/5 : DEFENSIVE - we are going to add some defensive stats together, Defensive Rebounds per game, Steals per game, Blocks per game
# Then we are going to subtract the Fouls per game

df2['Def_Par'] = df2['DREB_PG'] + df2['STL_PG'] + df2['BLK_PG'] - df2 ['PF_PG']

df2[["PlayerName", "Def_Par", ]].sort_values(by="Def_Par", ascending=False).head(10).round(1)

,PlayerName,Def_Par
25,Anthony Davis,10.1
452,Rudy Gobert,8.8
262,Joel Embiid,8.7
406,Nikola Jokic,8.7
139,Domantas Sabonis,8.1
173,Giannis Antetokounmpo,8.1
348,Luka Doncic,8.0
32,Bam Adebayo,7.8
340,LeBron James,7.1
159,Evan Mobley,7.0


In [13]:
# Parameter 4/5 : EFFICIENCY - we are going to calculate the True Shooting Percentage using the formula 

df2["TS%"] = (df2["PPG"] / (2 * (df2["FGA_PG"] + 0.44 * df2["FTA_PG"]))*100)

df2[["PlayerName", "TS%"]].sort_values(by="TS%", ascending=False).head(10).round(1)

# To take in consideration players that are involved the most in the team's offense we are going to multiply this by the Usage percentage

df2['Eff_Par'] = df2['TS%'] * df2['USG%']

df2[["PlayerName", "Eff_Par"]].sort_values(by="Eff_Par", ascending=False).head(10).round(1)

,PlayerName,Eff_Par
262,Joel Embiid,25.2
348,Luka Doncic,22.4
173,Giannis Antetokounmpo,21.3
470,Shai Gilgeous-Alexander,21.0
478,Stephen Curry,19.7
406,Nikola Jokic,19.4
141,Donovan Mitchell,19.1
26,Anthony Edwards,18.9
133,Devin Booker,18.8
314,Kevin Durant,18.7


In [16]:
df2.columns

Index(['PlayerName', 'Team', 'Age', 'GP', 'W', 'L', 'W%', 'MIN_PG', 'FGM_PG',
       'FGA_PG', 'FG%', '3PM_PG', '3PA_PG', '3P%', 'FTM_PG', 'FTA_PG', 'FT%',
       'OREB_PG', 'DREB_PG', 'TREB_PG', 'AST_PG', 'TO_PG', 'STL_PG', 'BLK_PG',
       'BLKA_PG', 'PF_PG', 'PFD_PG', 'PPG', 'PlusMinus', 'NBAFantasyPoints',
       'DoubleDoubles', 'TripleDoubles', 'WNBAFantasyPoints', 'OFF_Rating',
       'DEF_Rating', 'Net_Rating', 'AST_Ratio', 'OREB%', 'DREB%', 'TREB%',
       'TOV%', 'USG%', 'Win_Par', 'P/M', 'Off_Par', 'OffRat_Par', 'Def_Par',
       'TS%', 'Eff_Par', 'FTMiss', 'FGMiss', 'PER'],
      dtype='object')

In [20]:
# Parameter 5/5 : IMPACT - we are going to calculate the Player Efficiency Rating

# To do so, we are going to use the 'linear weights PER', starting by assigning different coefficients for the stats we are going to need for PER

weights = {
    'FGM_PG': 85.910,
    '3PM_PG': 51.757,
    'FTM_PG': 46.845,
    'BLK_PG': 39.190,
    'OREB_PG': 39.190,
    'AST_PG': 34.677,
    'DREB_PG': 14.707,
    'PF_PG': 17.174,
    'FTMiss': 20.091,
    'FGMiss': 39.190,
    'TO_PG': 53.897
}

# Calculate misses
df2['FTMiss'] = df2['FTA_PG'] - df2['FTM_PG']
df2['FGMiss'] = df2['FGA_PG'] - df2['FGM_PG']

df2['Imp_Par'] = ((df2['FGM_PG'] * weights['FGM_PG']) +
                 (df2['3PM_PG'] * weights['3PM_PG']) +
                 (df2['FTM_PG'] * weights['FTM_PG']) +
                 (df2['BLK_PG'] * weights['BLK_PG']) +
                 (df2['OREB_PG'] * weights['OREB_PG']) +
                 (df2['AST_PG'] * weights['AST_PG']) +
                 (df2['DREB_PG'] * weights['DREB_PG']) -
                 (df2['PF_PG'] * weights['PF_PG']) -
                 (df2['FTMiss'] * weights['FTMiss']) -
                 (df2['FGMiss'] * weights['FGMiss']) -
                 (df2['TO_PG'] * weights['TO_PG'])) / df2['MIN_PG']


df2[['PlayerName', 'Imp_Par', 'Eff_Par', 'OffRat_Par', 'Def_Par', 'Win_Par' ]].sort_values(by='Imp_Par', ascending=False).head(20)

,PlayerName,Imp_Par,Eff_Par,OffRat_Par,Def_Par,Win_Par
262,Joel Embiid,40.612876,25.217962,21.10745,8.7,4.77
406,Nikola Jokic,37.036038,19.411530,22.71905,8.7,5.66
173,Giannis Antetokounmpo,35.297994,21.300885,22.31215,8.1,5.02
470,Shai Gilgeous-Alexander,34.984919,21.011755,21.35585,5.2,5.88
348,Luka Doncic,33.878210,22.396205,20.25075,8.0,3.49
515,Tyrese Haliburton,32.479349,16.280570,21.55250,4.1,2.98
25,Anthony Davis,30.374437,16.184974,16.09190,10.1,3.38
314,Kevin Durant,28.207396,18.706999,17.94855,6.2,3.41
339,Lauri Markkanen,27.944511,15.953846,17.16190,5.6,2.52
139,Domantas Sabonis,27.770826,14.628477,17.31900,8.1,4.04


In [30]:
# Let's take a look at the totals

df2['Totals'] = df2['Imp_Par'] + df2['Eff_Par'] + df2['OffRat_Par'] + df2['Def_Par'] + df2['Win_Par']

df2[['PlayerName', 'Totals']].sort_values(by='Totals', ascending=False).head(10).round(2)

,PlayerName,Totals
262,Joel Embiid,100.41
406,Nikola Jokic,93.53
173,Giannis Antetokounmpo,92.03
470,Shai Gilgeous-Alexander,88.43
348,Luka Doncic,88.02
515,Tyrese Haliburton,77.39
25,Anthony Davis,76.13
248,Jayson Tatum,74.86
314,Kevin Durant,74.47
301,Kawhi Leonard,72.74


In [31]:
df2.to_csv('5Parameters.csv', index=False)